In [3]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import io
# import imageio
from IPython.display import Image, display
from ipywidgets import widgets, Layout, HBox

import pandas as pd
from tensorflow.python.keras.models import Sequential,Model
from tensorflow.python.keras.layers import Dense, Dropout, BatchNormalization,Input, LSTM
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.python.keras import regularizers,initializers
import keras.backend as Kr


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
# Library for Gaussian process
# import GPy
##Library for visualization
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib;matplotlib.rcParams['figure.figsize'] = (10,10)
import pylab 
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from tqdm import tqdm

In [ ]:
df = pd.read_csv("training_data.csv")
model_interpolation = keras.models.load_model('model_real.h5')

In [ ]:
### generate locations
s_loc =np.empty((0,3))
for i in range(len(np.unique(df["t"]))):
    
    loc1 = 0.7095714
    loc2 = 0.22975
    n = 8
    N = int(n**2) ## sample size
    M = 1 ## Number of replicate
    coord1 = np.linspace((loc1-0.2),(loc1+0.2),n)
    coord2 = np.linspace((loc2-0.2),(loc2+0.2),n)
    s1,s2 = np.meshgrid(coord1,coord2)
    s_a = np.vstack((s1.flatten(),s2.flatten())).T

    ## temporal embedding

    t = np.array([i+1]*N)

    s_b = t/len(np.unique(df["t"]))
    s_b = s_b.reshape(N,1)
    s_a = np.hstack((s_a,s_b))
    s_loc = np.append(s_loc,s_a,axis=0)


In [ ]:
## Spatial co-ordinate embedding
## interpolation in the neighbourhood of a certain location 
N = len(s_loc)
s = s_loc[:,:2]

num_basis = [5**2,9**2,11**2]
knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
##Wendland kernel
K = 0
phi = np.zeros((N, sum(num_basis)))
for res in range(len(num_basis)):
    theta = 1/np.sqrt(num_basis[res])*2.5
    knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
    knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
    for i in range(num_basis[res]):
        d = np.linalg.norm(s-knots[i,:],axis=1)/theta
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi[j,i + K] = 0
    K = K + num_basis[res]


## temporal embedding

s = s_loc[:,2]

## time basis 
num_basis = [10,15,45]
std_arr = [0.4,0.2,0.1]
#std_arr = [0.3,0.15,0.05]
mu_knots = [np.linspace(0,1,int(i)) for i in num_basis]

phi_t = np.zeros((N, sum(num_basis)))
K = 0
for res in range(len(num_basis)):
    std = std_arr[res]
    for i in range(num_basis[res]):
        d = np.square(np.absolute(s-mu_knots[res][i]))
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi_t[j,i + K] = np.exp(-0.5 * d[j]/(std**2))
            else:
                phi_t[j,i + K] = 0
    K = K + num_basis[res]

In [ ]:
phi2 = np.hstack((phi_t,phi))
phi2.shape

In [ ]:
pred_z = model_interpolation.predict(phi2)

In [ ]:
s_loc = np.append(s_loc,pred_z.reshape((N,1)),axis = 1)

In [ ]:
t_arr = np.array(range(1,1097,1))
t_arr = t_arr/1096

In [ ]:
## change the i accordingly 
## run from here

dim = n
data = np.empty((0,20,dim,dim))
for i in tqdm(range(1096)):
    if i == 1046:
        break
    else:
        data_i = np.empty((0,dim,dim))
#         z`print(data_i.shape)
        for j in range(20):
            b = s_loc[s_loc[:, 2] == t_arr[i+j]]
#             b[:,3] = (b[:,3] - min(b[:,3]))/(max(b[:,3]) - min(b[:,3]))
            val = b[:,3].reshape((dim,dim)).T
            val = val.reshape((1,dim,dim))
            data_i = np.append(data_i,val,axis = 0)
#         print(data_i.shape)
        data_i = data_i.reshape((1,20,dim,dim))
        data = np.append(data,data_i,axis = 0)
            

In [ ]:
### reshape the prediction in this format
data.shape

In [ ]:
### create variable y
loc1 = 0.75
loc2 = 0.65

N1 = len(np.unique(df["t"]))
s = np.stack((np.array([loc1]*len(np.unique(df["t"]))),np.array([loc2]*len(np.unique(df["t"]))))).T
num_basis = [5**2,9**2,11**2]
knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
##Wendland kernel
K = 0
phi = np.zeros((N1, sum(num_basis)))
for res in range(len(num_basis)):
    theta = 1/np.sqrt(num_basis[res])*2.5
    knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
    knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
    for i in range(num_basis[res]):
        d = np.linalg.norm(s-knots[i,:],axis=1)/theta
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi[j,i + K] = 0
    K = K + num_basis[res]

## temporal embedding

s = np.unique(s_loc[:,2])

## time basis 
num_basis = [10,15,45]
std_arr = [0.4,0.2,0.1]
#std_arr = [0.3,0.15,0.05]
mu_knots = [np.linspace(0,1,int(i)) for i in num_basis]

phi_t = np.zeros((N1, sum(num_basis)))
K = 0
for res in range(len(num_basis)):
    std = std_arr[res]
    for i in range(num_basis[res]):
        d = np.square(np.absolute(s-mu_knots[res][i]))
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi_t[j,i + K] = np.exp(-0.5 * d[j]/(std**2))
            else:
                phi_t[j,i + K] = 0
    K = K + num_basis[res]

phi2 = np.hstack((phi_t,phi))
phi2.shape

In [ ]:
y = model_interpolation.predict(phi2)

In [ ]:
plt.plot(range(1096),y)

In [ ]:
y_arr = np.empty((0,30))

for i in tqdm(range(20,1066,1)):
    arr_i = y[i:i+30].T
#     print(arr_i.shape)
    y_arr = np.append(y_arr,arr_i,axis = 0)
y_arr.shape    

In [ ]:
# # Download and load the dataset.
# fpath = keras.utils.get_file(
#     "moving_mnist.npy",
#     "http://www.cs.toronto.edu/~nitish/unsupervised_video/mnist_test_seq.npy",
# )
# dataset = np.load(fpath)

In [ ]:
# Swap the axes representing the number of frames and number of data samples.
# dataset = np.swapaxes(dataset, 0, 1)
# # We'll pick out 1000 of the 10000 total examples and use those.
# dataset = dataset[:1000, ...]
# # Add a channel dimension since the images are grayscale.
# dataset = np.expand_dims(dataset, axis=-1)

In [ ]:
dataset = data
dataset = dataset.reshape((data.shape[0],data.shape[1],data.shape[2],data.shape[3],1))
dataset.shape

In [ ]:
# Split into train and validation sets using indexing to optimize memory.
indexes = np.arange(dataset.shape[0])
np.random.shuffle(indexes)
train_index = indexes[: int(0.95 * dataset.shape[0])]
val_index = indexes[int(0.95 * dataset.shape[0]) :]
train_dataset = dataset[train_index]
val_dataset = dataset[val_index]

# Normalize the data to the 0-1 range.
train_dataset = train_dataset 
val_dataset = val_dataset

# We'll define a helper function to shift the frames, where
# `x` is frames 0 to n - 1, and `y` is frames 1 to n.
# def create_shifted_frames(data):
#     x = data[:, 0 : data.shape[1] - 1, :, :]
#     y = data[:, 1 : data.shape[1], :, :]
#     return x, y


# Apply the processing function to the datasets.
x_train, y_train = dataset[train_index],y_arr[train_index]
x_val, y_val = dataset[val_index],y_arr[val_index]

# Inspect the dataset.
print("Training Dataset Shapes: " + str(x_train.shape) + ", " + str(y_train.shape))
print("Validation Dataset Shapes: " + str(x_val.shape) + ", " + str(y_val.shape))

In [ ]:
# Construct a figure on which we will visualize the images.
fig, axes = plt.subplots(5, 4, figsize=(10, 8))

# Plot each of the sequential images for one random data example.
data_choice = np.random.choice(range(len(x_train)), size=1)[0]
# data_choice = 5
for idx, ax in enumerate(axes.flat):
    ax.imshow(np.squeeze(train_dataset[data_choice][idx]), cmap="gray")
    ax.set_title(f"Frame {idx + 1}")
    ax.axis("off")

# Print information and display the figure.
print(f"Displaying frames for example {data_choice}.")
plt.show()

In [ ]:
# Construct the input layer with no definite frame size.
inp = layers.Input(shape=(20,8,8,1))

# We will construct 3 `ConvLSTM2D` layers with batch normalization,
# followed by a `Conv3D` layer for the spatiotemporal outputs.
x = layers.ConvLSTM2D(
    filters=64,
    kernel_size=(5, 5),
    padding="same",
    return_sequences=True,
    activation="relu",
)(inp)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(
    filters=64,
    kernel_size=(3, 3),
    padding="same",
    return_sequences=True,
    activation="relu",
)(x)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(
    filters=64,
    kernel_size=(1, 1),
    padding="same",
    return_sequences=True,
    activation="relu",
)(x)
# x = layers.Conv3D(
#     filters=1, kernel_size=(3, 3, 3), activation="sigmoid", padding="same"
# )(x)

x = layers.Flatten()(x)
x = layers.Dense(30, activation = "linear")(x)

# Next, we will build the complete model and compile it.
model = keras.models.Model(inp, x)
model.compile(
    loss=keras.losses.mae, optimizer=keras.optimizers.Adam(),
)

In [ ]:
# Define some callbacks to improve training.
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5)

# Define modifiable training hyperparameters.
epochs = 25
batch_size = 5

# Fit the model to the training data.
model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_val, y_val),
    callbacks=[early_stopping, reduce_lr])

In [ ]:
## change the i accordingly 
## run from here

dim = n
data_test = np.empty((0,20,dim,dim))
for i in tqdm(range(1096)):
    if i == 1076:
        break
    else:
        data_i = np.empty((0,dim,dim))
#         z`print(data_i.shape)
        for j in range(20):
            b = s_loc[s_loc[:, 2] == t_arr[i+j]]
#             b[:,3] = (b[:,3] - min(b[:,3]))/(max(b[:,3]) - min(b[:,3]))
            val = b[:,3].reshape((dim,dim)).T
            val = val.reshape((1,dim,dim))
            data_i = np.append(data_i,val,axis = 0)
#         print(data_i.shape)
        data_i = data_i.reshape((1,20,dim,dim))
        data_test = np.append(data_test,data_i,axis = 0)

In [ ]:
data_test = data_test.reshape((data_test.shape[0],data_test.shape[1],data_test.shape[2],data_test.shape[3],1))

In [ ]:
pred_y = model.predict(data_test)

In [ ]:
pred_y.shape
pred_y_arr = np.zeros((1105))

for i in range(len(pred_y)-1):
    pred_y_arr[i] = pred_y[i,0]
pred_y_arr[1075:1105] = pred_y[1075]

In [ ]:
orig_y = np.array([0.03]*30)

In [ ]:
## save this data for plotting
plt.plot(range(1000,1096,1),y[1000:])
plt.plot(range(1000,1125,1),pred_y_arr[980:])

In [ ]:
# Select a random example from the validation dataset.
example = val_dataset[np.random.choice(range(len(val_dataset)), size=1)[0]]

# Pick the first/last ten frames from the example.
frames = example[:10, ...]
original_frames = example[10:, ...]

# Predict a new set of 10 frames.
for _ in range(10):
    # Extract the model's prediction and post-process it.
    new_prediction = model.predict(np.expand_dims(frames, axis=0))
    new_prediction = np.squeeze(new_prediction, axis=0)
    predicted_frame = np.expand_dims(new_prediction[-1, ...], axis=0)

    # Extend the set of prediction frames.
    frames = np.concatenate((frames, predicted_frame), axis=0)

# Construct a figure for the original and new frames.
fig, axes = plt.subplots(2, 10, figsize=(20, 4))

# Plot the original frames.
for idx, ax in enumerate(axes[0]):
    print(idx)
    ax.imshow(np.squeeze(original_frames[idx]), cmap="gray")
    ax.set_title(f"Frame {idx + 11}")
    ax.axis("off")

# Plot the new frames.
new_frames = frames[10:, ...]
for idx, ax in enumerate(axes[1]):
    ax.imshow(np.squeeze(new_frames[idx]), cmap="gray")
    ax.set_title(f"Frame {idx + 11}")
    ax.axis("off")

# Display the figure.
plt.show()